In [1]:
# Update sklearn to prevent version mismatches

#!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=c773649d0350068599c9ec9ccf656b071efe5b58c413c1b4021bca1d9a2a98cf
  Stored in directory: c:\users\gloam\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [1]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 

#!pip install joblib

In [1]:
#Import Dependencies
import warnings
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
df

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,FALSE POSITIVE,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,...,-152,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
6987,FALSE POSITIVE,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,...,-166,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
6988,CANDIDATE,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,...,-220,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
6989,FALSE POSITIVE,0,0,1,0,0.681402,2.430000e-06,-2.430000e-06,132.181750,0.002850,...,-236,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [4]:
# Drop the null columns where all values are null
data = df.dropna(axis='columns', how='all')
# Drop the null rows
data = df.dropna()
data.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
# Convert dtypes of int64 to float64
for column, content in data.items():
    if data[column].dtype == 'int64':
        data = data.astype({column: 'float64'})

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values. **NO COLUMNS WERE DELETED, WAS NOT SURE WHAT WAS GOING TO BE USED

#selected_features = df[['names', 'of', 'selected', 'features', 'here']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
# Assign data to X and y
X = data.drop("koi_disposition", axis=1)
y = data["koi_disposition"]

In [8]:
# Split data into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1.0,0.0,0.0,0.0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,...,-286.0,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0.0,0.0,0.0,0.0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,...,-72.0,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0.0,1.0,0.0,0.0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,...,-159.0,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1.0,0.0,0.0,0.0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,...,-146.0,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0.0,1.0,1.0,1.0,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,...,-167.0,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


In [10]:
X_train.shape

(5243, 40)

In [12]:
y_train.shape

(5243,)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [13]:
# Scale X values
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Build the Model



In [14]:
#Train the model 
model_1 = LogisticRegression(solver='newton-cg', multi_class='auto')
model_1.fit(X_train_scaled, y_train)

model_1_training_score = round(model_1.score(X_train_scaled, y_train)*100,3)
base_accuracy = round(model_1.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {model_1_training_score} %")
print(f"Testing Data Score: {base_accuracy} %")

Training Data Score: 85.504 %
Testing Data Score: 86.213 %


In [15]:
#Select Features. Evaluate features
feature_names = X.columns.tolist()
selector = RFECV(estimator=model_1, cv=5, step=1)
_ = selector.fit(X_train_scaled, y_train)

In [16]:
# Determine which features ought to be kept
preSelected_features = sorted(zip(selector.ranking_, feature_names))
ranked_features = pd.DataFrame(preSelected_features, columns=['Ranking', 'Feature'])
ranked_features = ranked_features.set_index('Feature')
ranked_features.head()

,Ranking
Feature,
dec,1
koi_depth,1
koi_duration,1
koi_duration_err1,1
koi_duration_err2,1


In [18]:
preSelected_features

[(1, 'dec'),
 (1, 'koi_depth'),
 (1, 'koi_duration'),
 (1, 'koi_duration_err1'),
 (1, 'koi_duration_err2'),
 (1, 'koi_fpflag_co'),
 (1, 'koi_fpflag_ec'),
 (1, 'koi_fpflag_nt'),
 (1, 'koi_fpflag_ss'),
 (1, 'koi_impact'),
 (1, 'koi_impact_err1'),
 (1, 'koi_impact_err2'),
 (1, 'koi_model_snr'),
 (1, 'koi_period'),
 (1, 'koi_period_err1'),
 (1, 'koi_period_err2'),
 (1, 'koi_slogg'),
 (1, 'koi_slogg_err1'),
 (1, 'koi_slogg_err2'),
 (1, 'koi_srad'),
 (1, 'koi_srad_err1'),
 (1, 'koi_srad_err2'),
 (1, 'koi_steff'),
 (1, 'koi_steff_err1'),
 (1, 'koi_steff_err2'),
 (1, 'koi_tce_plnt_num'),
 (1, 'koi_teq'),
 (1, 'koi_time0bk'),
 (1, 'koi_time0bk_err1'),
 (1, 'koi_time0bk_err2'),
 (1, 'ra'),
 (2, 'koi_kepmag'),
 (3, 'koi_insol_err1'),
 (4, 'koi_prad_err1'),
 (5, 'koi_prad'),
 (6, 'koi_insol'),
 (7, 'koi_insol_err2'),
 (8, 'koi_prad_err2'),
 (9, 'koi_depth_err2'),
 (10, 'koi_depth_err1')]

In [19]:
# Remove features with Ranking > 16
selected_features = []
for tup in preSelected_features:
    if tup[0] < 17:
        selected_features.append(tup[1])

In [20]:
# Use new data for all subsequent models
## Assign new data to X 
X_train_select = X_train[selected_features]
X_test_select = X_test[selected_features]

X_scaler = MinMaxScaler().fit(X_train_select)
X_train_scaled = X_scaler.transform(X_train_select)
X_test_scaled = X_scaler.transform(X_test_select)

## Train new model
model_2 = LogisticRegression(solver='newton-cg', multi_class='auto')
model_2.fit(X_train_scaled, y_train)

model_2_training_score = round(model_2.score(X_train_scaled, y_train)*100,3)
select_features_accuracy = round(model_2.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {model_2_training_score} %")
print(f"Testing Data Score: {select_features_accuracy} %")

Training Data Score: 85.504 %
Testing Data Score: 86.213 %


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [21]:
# Create the GridSearchCV model
model_3 = LogisticRegression(solver='newton-cg', multi_class='auto')

param_grid = {
    'C': np.logspace(0, 4, 10),
    'penalty': ['l2']
}
grid = GridSearchCV(model_3, param_grid, cv=5, verbose=0)

# Train the model with GridSearch
_ = grid.fit(X_train_scaled, y_train)

In [22]:
# Train the model with GridSearch
# Tuned parameters
C = grid.best_params_['C']
penalty = grid.best_params_['penalty']

# Tuned model
tuned_model = LogisticRegression(solver='newton-cg', multi_class='auto',
                                 C=C, penalty=penalty)
tuned_model.fit(X_train_scaled, y_train)

model_3_training_score = round(tuned_model.score(X_train_scaled, y_train)*100,3)
tuned_accuracy = round(tuned_model.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {model_3_training_score} %")
print(f"Testing Data Score: {tuned_accuracy} %")

Training Data Score: 88.709 %
Testing Data Score: 89.474 %


In [23]:
print(grid.best_score_)

0.8842261623210763


# MODEL PREDICTIONS AND EVALUATIONS

In [24]:
#PREDICTIONS
predictions = tuned_model.predict(X_test_scaled)
classifications = y_test.unique().tolist()

prediction_actual = {
    'Actual': y_test,
    'Prediction': predictions
}

PA_df = pd.DataFrame(prediction_actual)
PA_df = PA_df.set_index('Actual').reset_index()
PA_df.head(15)

,Actual,Prediction
0,CANDIDATE,CANDIDATE
1,FALSE POSITIVE,FALSE POSITIVE
2,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE
4,CANDIDATE,CANDIDATE
5,FALSE POSITIVE,FALSE POSITIVE
6,CANDIDATE,CANDIDATE
7,FALSE POSITIVE,FALSE POSITIVE
8,FALSE POSITIVE,FALSE POSITIVE
9,FALSE POSITIVE,FALSE POSITIVE


In [28]:
#EVALUATIONS
evaluations = {'': ['Base Model', 'Select Features Model', 'Tuned Model'],
               'Accuracy': [f"{base_accuracy}%", f"{select_features_accuracy}%", f"{tuned_accuracy}%"]}

evaluations_df = pd.DataFrame(evaluations)
evaluations_df = evaluations_df.set_index('')

evaluations_df.to_csv('output/LogisticRegression_evaluate.csv')
evaluations_df

,Accuracy
,
Base Model,86.213%
Select Features Model,86.213%
Tuned Model,89.474%


# Save the Model

In [30]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

import joblib
filename = ('models/EPLogisticRegression_evaluate.sav')
joblib.dump(grid,filename)

['models/EPLogisticRegression_evaluate.sav']